In [2]:
import geemap
import ee
import os
import geopandas


In [3]:
ee.Authenticate()
ee.Initialize(project='digital-yeti-417904')
print('Earth Engine Initialized')

Earth Engine Initialized


In [4]:
brazil_shapefile = ee.FeatureCollection('FAO/GAUL/2015/level0')\
                      .filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
brazil_shapefile_geo = brazil_shapefile.geometry()

In [28]:
## Burn datasets ##
burned_boundaries = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")

burned_images = ee.ImageCollection("MODIS/061/MCD64A1")
burned_images = burned_images.filter(ee.Filter.date('2017-01-01', '2018-01-01'))

burnedArea = burned_images.select('BurnDate')
burnedArea = burnedArea.mean()
burnedArea = burnedArea.clip(brazil_shapefile)

# geemap.ee_export_image(burnedArea, filename='file.tif', scale=90, region=brazil_shapefile_geo)
# Get image pixel size without downloading it
# print(burnedArea.getInfo().keys())
# print(burnedArea.getInfo()['bands'][0]['dimensions'])

Map = geemap.Map()

# Protected Areas ##
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")
brazil_protected_areas = protected_areas.filterBounds(brazil_shapefile)

Map.addLayer(brazil_protected_areas, {'color': '00FF00', 'fillColor': '80FF0080'}, 'Protected Area')
Map.addLayer(brazil_shapefile, {'color': '808080', 'fillColor': '80808080'}, 'Border')

Map.add_layer(burnedArea, {'palette': ['red']}, 'Burned Area')
Map.add_layer_control()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
def clip_image(image):
    return image.clip(brazil_shapefile)



## Population Datasets ##
population = ee.ImageCollection('WorldPop/GP/100m/pop')
population = population.filter(ee.Filter.date('2017-01-01', '2018-01-01')).mean()

population = population.clip(brazil_shapefile)

# print(population.getInfo()['bands'][0]['dimensions'])
print(population.getInfo()['bands'])
# population = population.clip(brazil_shapefile)
# Map = geemap.Map()
# Map.add_layer(population)
# Map.add_layer_control()
# Map

[{'id': 'population', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]


In [7]:
start_of_2020 = '2020-01-01'
end_of_2020 = '2020-12-31'

fire = (ee.ImageCollection('MODIS/006/MCD64A1')
        .filter(ee.Filter.date(start_of_2020, end_of_2020))).max()

population = (ee.ImageCollection('WorldPop/GP/100m/pop')
              .filter(ee.Filter.date(start_of_2020, end_of_2020)).mean())




# Get the projection info from the population dataset
pop_proj = population.projection()

# Use the same projection for the fire data
# fire_proj = fire.select('BurnDate').reproject(pop_proj)

# Create a binary mask of the 'BurnDate' band indicating whether a wildfire occurred
fire_mask_partial = fire.select('BurnDate').gt(0)

# Use 'where' to assign population counts to burn areas
fire_mask_full = ee.Image.constant(0).where(fire_mask_partial, 1)
pop_affected = population.where(fire_mask_full.Not(), 0)

# Compute the total population affected by wildfires

# brazil_simplified = brazil_shapefile.simplify(maxError=10000)
geometry = brazil_shapefile_geo.simplify(maxError=5000)
# Make geometry all world

fire = fire.clip(brazil_shapefile)
pop_affected = pop_affected.clip(brazil_shapefile)
fire_mask_full = fire_mask_full.clip(brazil_shapefile)
fire_mask_partial = fire_mask_partial.clip(brazil_shapefile)

total_pop_affected = pop_affected.reduceRegion(reducer=ee.Reducer.sum(), geometry=geometry, scale=100, maxPixels=1e10)

# Obtain result
# result = total_pop_affected.getInfo()
# print('Total population affected by wildfires in 2020: ', result['population'])



Map = geemap.Map()
vis = {'min': 0, 'max': 1, 'palette': ['green', 'red'], 'bands': ['population']}
Map.addLayer(pop_affected, vis, 'Pop')
vis = {'min': 0, 'max': 1, 'palette': ['green', 'red'], 'bands': ['population']}
Map.addLayer(population, vis, 'Pop')
vis = {'min': 0, 'max': 1, 'palette': ['blue', 'orange'], 'bands': ['BurnDate']}
Map.addLayer(fire_mask_partial, vis, 'partial fire mask')
# vis = {'min': 0, 'max': 1, 'palette': ['green', 'red']}
# Map.addLayer(fire_mask_full, vis, 'full fire mask')

Map.addLayerControl()
Map




# +1 315 766 4675 jimmy

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
total_fire_area = not_fire_mask.reduceRegion(reducer=ee.Reducer.sum(), geometry=geometry, scale=100, maxPixels=1e10)
total_fire_area.getInfo()

NameError: name 'not_fire_mask' is not defined

In [ ]:
mask = population.eq(0)

# Use 'where' to assign population counts to burn areas
pop_affected = population.where(mask, 0)

total_pop = pop_affected.reduceRegion(reducer=ee.Reducer.sum(), geometry=geometry, scale=100, maxPixels=1e10)
result = total_pop.getInfo()
result['population'] / 1e6

In [ ]:
print('CRS: ', fire_proj.getInfo()['bands'][0])
print(pop_proj.nominalScale())
